In [3]:
import pandas as pd
import numpy as np
import scipy
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from copy import deepcopy
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.interpolate import CubicSpline
from sklearn.metrics import mean_squared_error
import warnings
import itertools
warnings.filterwarnings("ignore")
import arviz as az
import scipy
import matplotlib.patches as patches
import statsmodels.stats.api as sms
import scipy.stats.kde as kde

# Read 16S data

In [4]:
df_16S = pd.read_excel('../../../../../data/16S_absolute_abundance.xlsx', index_col=0)
df_16S.columns = [c.replace('/','_slash_').replace(' ','_space_').replace('[','_leftsquarebracket_').replace(']','_rightsquarebracket_').replace('-','_dash_').replace('.','_dot_').replace('(','_leftroundbracket').replace(')','_rightroundbracket_') for c in df_16S.columns]
df_16S.head()

,A2,ASF356,Acetatifactor,Acetatifactor_dash_muris,Acidovorax,Acinetobacter,Acinetobacter_dash_sp_dot__dash_CIP_dash_56_dot_2,Acutalibacter_dash_muris,Adlercreutzia_dash_mucosicola,Adlercreutzia_dash_muris,...,_leftsquarebracket_Eubacterium_rightsquarebracket__dash_ventriosum_dash_group,_leftsquarebracket_Eubacterium_rightsquarebracket__dash_xylanophilum_dash_group,_leftsquarebracket_Ruminococcus_rightsquarebracket__dash_gnavus_dash_group,_leftsquarebracket_Ruminococcus_rightsquarebracket__dash_torques_dash_group,bacterium_dash_enrichment_dash_culture_dash_clone_dash_M153,gut_dash_metagenome,human_dash_gut_dash_metagenome,marine_dash_metagenome,metagenome,mouse_dash_gut_dash_metagenome
SampleID,,,,,,,,,,,,,,,,,,,,,
sample131,0.0,108301.701888,7101.750943,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,63028.039623,0.0,0.0,0.0,165115.709436,0.0,0.0,0.0,1.322701e+05
sample132,0.0,405068.034693,50064.588558,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,80103.341692,0.0,0.0,0.0,106501.033841,0.0,0.0,0.0,1.003112e+06
sample133,0.0,36169.603950,45332.570284,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,221840.237562,0.0,0.0,0.0,1.866352e+05
sample134,0.0,484302.054587,10150.043060,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,105850.449056,0.0,0.0,0.0,919303.900024,0.0,0.0,0.0,8.714537e+05
sample135,0.0,91489.970533,42374.302141,0.0,0.0,0.0,0.0,0.0,12519.680178,0.0,...,0.0,0.000000,0.0,0.0,0.0,581683.602124,0.0,0.0,0.0,5.335310e+05


# Add pseudocount for zeros

In [5]:
for sample_id in df_16S.index:
    sample = np.array(df_16S.loc[sample_id])
    minval = np.min(sample[np.nonzero(sample)]) # minimum non-zero value
    sample[sample==0] = minval
    df_16S.loc[sample_id] = sample
df_16S.head()

,A2,ASF356,Acetatifactor,Acetatifactor_dash_muris,Acidovorax,Acinetobacter,Acinetobacter_dash_sp_dot__dash_CIP_dash_56_dot_2,Acutalibacter_dash_muris,Adlercreutzia_dash_mucosicola,Adlercreutzia_dash_muris,...,_leftsquarebracket_Eubacterium_rightsquarebracket__dash_ventriosum_dash_group,_leftsquarebracket_Eubacterium_rightsquarebracket__dash_xylanophilum_dash_group,_leftsquarebracket_Ruminococcus_rightsquarebracket__dash_gnavus_dash_group,_leftsquarebracket_Ruminococcus_rightsquarebracket__dash_torques_dash_group,bacterium_dash_enrichment_dash_culture_dash_clone_dash_M153,gut_dash_metagenome,human_dash_gut_dash_metagenome,marine_dash_metagenome,metagenome,mouse_dash_gut_dash_metagenome
SampleID,,,,,,,,,,,,,,,,,,,,,
sample131,887.718868,108301.701888,7101.750943,887.718868,887.718868,887.718868,887.718868,887.718868,887.718868,887.718868,...,887.718868,63028.039623,887.718868,887.718868,887.718868,165115.709436,887.718868,887.718868,887.718868,1.322701e+05
sample132,910.265247,405068.034693,50064.588558,910.265247,910.265247,910.265247,910.265247,910.265247,910.265247,910.265247,...,910.265247,80103.341692,910.265247,910.265247,910.265247,106501.033841,910.265247,910.265247,910.265247,1.003112e+06
sample133,482.261386,36169.603950,45332.570284,482.261386,482.261386,482.261386,482.261386,482.261386,482.261386,482.261386,...,482.261386,482.261386,482.261386,482.261386,482.261386,221840.237562,482.261386,482.261386,482.261386,1.866352e+05
sample134,2900.012303,484302.054587,10150.043060,2900.012303,2900.012303,2900.012303,2900.012303,2900.012303,2900.012303,2900.012303,...,2900.012303,105850.449056,2900.012303,2900.012303,2900.012303,919303.900024,2900.012303,2900.012303,2900.012303,8.714537e+05
sample135,1926.104643,91489.970533,42374.302141,1926.104643,1926.104643,1926.104643,1926.104643,1926.104643,12519.680178,1926.104643,...,1926.104643,1926.104643,1926.104643,1926.104643,1926.104643,581683.602124,1926.104643,1926.104643,1926.104643,5.335310e+05


# Read metadata for rs and control

In [6]:
df_meta = pd.read_excel('../../../../../data/meta_data.xlsx', index_col=0)
df_meta = df_meta[(df_meta.Diet=='Resistant starch') | (df_meta.Diet=='Control')]
df_meta['Diet'] = [1 if g=='Resistant starch' else 0 for g in df_meta['Diet']]
df_meta.head()

,MiceID,Vendor,Day,Diet
SampleID,,,,
sample131,BC1,Beijing,0,0
sample132,BC2,Beijing,0,0
sample133,BC3,Beijing,0,0
sample134,BC4,Beijing,0,0
sample135,BC5,Beijing,0,0


# Select the 20 most abundant taxa

In [7]:
topX = 20
df_16S_T = df_16S.loc[df_meta.index].T
df_16S_T['mean'] = df_16S_T.mean(axis=1)
df_16S_T = df_16S_T.sort_values(by=['mean'],axis=0,ascending=False)
df_16S_T = df_16S_T.drop('mean', axis=1)
df_16S_topX = df_16S_T.iloc[0:topX].T
df_16S_topX.head()

,Muribaculaceae,Faecalibaculum,Bacteroides_dash_acidifaciens,Lachnospiraceae_dash_NK4A136_dash_group,Parabacteroides_dash_goldsteinii,Bacteroides,Lachnospiraceae,Alloprevotella,Lachnospiraceae_dash_bacterium_dash_28_dash_4,Desulfovibrionaceae,Parasutterella,Akkermansia_dash_muciniphila,Oscillospiraceae,Rikenellaceae_dash_RC9_dash_gut_dash_group,Lachnospiraceae_dash_bacterium_dash_10_dash_1,Ruminococcus_dash_sp_dot__dash_N15_dot_MGS_dash_57,Helicobacter_dash_ganmani,Mucispirillum_dash_schaedleri,Clostridia_dash_vadinBB60_dash_group,Alistipes
SampleID,,,,,,,,,,,,,,,,,,,,
sample131,2.178906e+07,2.175266e+07,1.682227e+06,2.719971e+06,103863.107548,3.826068e+05,3.380433e+06,9.717858e+06,1.012000e+05,1.819824e+05,2.213083e+06,3.771030e+06,6.249541e+05,6.779509e+06,674666.339629,7101.750943,5326.313208,2.008020e+06,470491.000005,1.064375e+06
sample132,2.539822e+07,1.732963e+07,4.455748e+06,3.721164e+06,59167.241023,2.645231e+06,4.563160e+06,2.849130e+05,7.898372e+06,9.912789e+05,3.591907e+06,2.610641e+06,3.932346e+06,3.418046e+06,90116.259404,10012.917712,11833.448205,1.345281e+07,142911.643701,1.247063e+06
sample133,1.379895e+07,1.250166e+07,1.339240e+06,7.354486e+05,46297.093056,7.802989e+05,7.508810e+05,1.040238e+06,2.083369e+05,1.504656e+05,4.753168e+06,3.729327e+06,5.290407e+05,5.057957e+06,21701.762370,4822.613860,5304.875246,4.054854e+06,175060.883119,1.064833e+06
sample134,3.867746e+07,2.484296e+07,1.024574e+07,4.120917e+06,184150.781235,4.157168e+06,8.172235e+06,3.694616e+06,1.566007e+05,2.321460e+06,4.982221e+06,2.262880e+07,5.088072e+06,2.312180e+07,511852.171464,21750.092272,20300.086120,5.086622e+06,194300.824295,8.069284e+06
sample135,1.994192e+07,1.935254e+07,4.775776e+06,1.094990e+06,72228.924105,1.040097e+06,1.251005e+06,1.814391e+06,6.067230e+04,2.725438e+05,3.164590e+06,4.727624e+06,5.441246e+05,1.064558e+07,42374.302141,1926.104643,6741.366250,7.617744e+05,45263.459106,2.844857e+06


In [8]:
list(df_16S_topX.columns)

['Muribaculaceae',
 'Faecalibaculum',
 'Bacteroides_dash_acidifaciens',
 'Lachnospiraceae_dash_NK4A136_dash_group',
 'Parabacteroides_dash_goldsteinii',
 'Bacteroides',
 'Lachnospiraceae',
 'Alloprevotella',
 'Lachnospiraceae_dash_bacterium_dash_28_dash_4',
 'Desulfovibrionaceae',
 'Parasutterella',
 'Akkermansia_dash_muciniphila',
 'Oscillospiraceae',
 'Rikenellaceae_dash_RC9_dash_gut_dash_group',
 'Lachnospiraceae_dash_bacterium_dash_10_dash_1',
 'Ruminococcus_dash_sp_dot__dash_N15_dot_MGS_dash_57',
 'Helicobacter_dash_ganmani',
 'Mucispirillum_dash_schaedleri',
 'Clostridia_dash_vadinBB60_dash_group',
 'Alistipes']

# Normalize max value of 16S data to 1

In [9]:
bacterial_taxa = list(df_16S_topX.columns)
df_16S_topX_w_meta = pd.merge(df_meta, df_16S_topX/df_16S_topX.max().max(), left_index=True, right_index=True, how='inner')
df_16S_topX_w_meta.head()

,MiceID,Vendor,Day,Diet,Muribaculaceae,Faecalibaculum,Bacteroides_dash_acidifaciens,Lachnospiraceae_dash_NK4A136_dash_group,Parabacteroides_dash_goldsteinii,Bacteroides,...,Parasutterella,Akkermansia_dash_muciniphila,Oscillospiraceae,Rikenellaceae_dash_RC9_dash_gut_dash_group,Lachnospiraceae_dash_bacterium_dash_10_dash_1,Ruminococcus_dash_sp_dot__dash_N15_dot_MGS_dash_57,Helicobacter_dash_ganmani,Mucispirillum_dash_schaedleri,Clostridia_dash_vadinBB60_dash_group,Alistipes
SampleID,,,,,,,,,,,,,,,,,,,,,
sample131,BC1,Beijing,0,0,0.121890,0.121687,0.009411,0.015216,0.000581,0.002140,...,0.012380,0.021096,0.003496,0.037925,0.003774,0.000040,0.000030,0.011233,0.002632,0.005954
sample132,BC2,Beijing,0,0,0.142080,0.096944,0.024926,0.020817,0.000331,0.014798,...,0.020094,0.014604,0.021998,0.019121,0.000504,0.000056,0.000066,0.075256,0.000799,0.006976
sample133,BC3,Beijing,0,0,0.077193,0.069936,0.007492,0.004114,0.000259,0.004365,...,0.026590,0.020862,0.002960,0.028295,0.000121,0.000027,0.000030,0.022683,0.000979,0.005957
sample134,BC4,Beijing,0,0,0.216366,0.138974,0.057316,0.023053,0.001030,0.023256,...,0.027871,0.126588,0.028463,0.129346,0.002863,0.000122,0.000114,0.028455,0.001087,0.045140
sample135,BC5,Beijing,0,0,0.111557,0.108260,0.026716,0.006125,0.000404,0.005818,...,0.017703,0.026447,0.003044,0.059553,0.000237,0.000011,0.000038,0.004261,0.000253,0.015914


# Calculate log-derivatives (dlog(B)/dt)

In [10]:
df_deriv = deepcopy(df_16S_topX_w_meta)
for curr_mice in set(df_deriv.MiceID):
    curr_df = df_deriv[df_deriv.MiceID==curr_mice]
    for taxon in bacterial_taxa:
        xdata = np.array(curr_df['Day'])
        ydata = np.array(curr_df[taxon])
        cs = CubicSpline(xdata, ydata)
        csd1 = cs.derivative(nu=1)
        ydata_d1 = csd1(xdata)
        df_deriv.loc[df_deriv.MiceID==curr_mice, taxon] = ydata_d1
df_deriv.head()

,MiceID,Vendor,Day,Diet,Muribaculaceae,Faecalibaculum,Bacteroides_dash_acidifaciens,Lachnospiraceae_dash_NK4A136_dash_group,Parabacteroides_dash_goldsteinii,Bacteroides,...,Parasutterella,Akkermansia_dash_muciniphila,Oscillospiraceae,Rikenellaceae_dash_RC9_dash_gut_dash_group,Lachnospiraceae_dash_bacterium_dash_10_dash_1,Ruminococcus_dash_sp_dot__dash_N15_dot_MGS_dash_57,Helicobacter_dash_ganmani,Mucispirillum_dash_schaedleri,Clostridia_dash_vadinBB60_dash_group,Alistipes
SampleID,,,,,,,,,,,,,,,,,,,,,
sample131,BC1,Beijing,0,0,-0.074386,-0.254233,-0.004071,-0.004919,0.000152,0.091566,...,-0.029396,-0.032129,0.017712,0.019519,-0.005393,0.000088,0.006910,-0.007715,-0.000913,0.039701
sample132,BC2,Beijing,0,0,-0.135226,-0.193439,-0.036332,0.017515,0.000315,-0.006180,...,-0.038113,-0.024804,-0.006381,0.011125,0.000221,-0.000053,0.093857,-0.093251,0.002053,0.009724
sample133,BC3,Beijing,0,0,-0.007883,-0.148905,-0.022818,0.000002,0.000183,0.069495,...,-0.065846,-0.021541,0.018099,0.010002,-0.001413,-0.000018,0.019876,-0.033317,0.021856,0.063697
sample134,BC4,Beijing,0,0,-0.021238,-0.148056,-0.078009,0.054405,-0.000708,-0.014161,...,-0.021315,-0.197212,0.027841,-0.042253,-0.002511,-0.000173,0.000063,-0.026145,0.009527,-0.045128
sample135,BC5,Beijing,0,0,0.149464,-0.060736,-0.009520,0.043082,0.000137,0.040215,...,-0.000796,-0.034933,0.009670,-0.026264,0.000484,0.000010,0.000023,0.007328,0.004431,0.022808


# Construct regression matrix

In [11]:
# Ymat should be samples by bacteria
Ymat = df_deriv[bacterial_taxa].values
Ymat = Ymat.flatten(order='F')
Ymat = StandardScaler().fit_transform(Ymat.reshape(-1,1)).reshape(1,-1)[0] # standardize
Ymat

array([-2.44202305, -4.44927117, -0.24797056, ...,  0.09420514,
        0.18872496,  0.10023815])

In [12]:
len(Ymat)

7200

In [13]:
Xmat = np.zeros(shape=(topX*len(df_deriv.index), (topX+2)*topX))
for k in np.arange(topX):
    Xmat[k*len(df_deriv.index):(k+1)*len(df_deriv.index),k*(topX+2)] = 1
    Xmat[k*len(df_deriv.index):(k+1)*len(df_deriv.index),k*(topX+2)+1] = df_deriv.Diet.values
    Xmat[k*len(df_deriv.index):(k+1)*len(df_deriv.index),k*(topX+2)+2:(k+1)*(topX+2)] = df_16S_topX_w_meta[bacterial_taxa].values
Xmat

array([[1.00000000e+00, 0.00000000e+00, 1.21890330e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 0.00000000e+00, 1.42080364e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 0.00000000e+00, 7.71927743e-02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        8.52639403e-04, 3.11541320e-03, 7.79673146e-03],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.77657663e-04, 1.00598652e-02, 1.67109239e-02],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        8.25431847e-05, 2.60386228e-03, 8.34436558e-03]])

# Write stan program

## write data

In [12]:
json_str = '{\n"N" : %d,\n'%(len(Ymat))

# y variable
json_str += '\"dlogX\" : [%s],\n'%(','.join(list(Ymat.astype(str))))

# x variable
for k1,c1 in enumerate(bacterial_taxa):
    # growth rate
    json_str += '\"growth_rate_%s\" : [%s],\n'%(c1,','.join(list(Xmat[:,k1*(topX+2)].astype(str))))
    # diet response
    json_str += '\"rs_response_%s\" : [%s],\n'%(c1,','.join(list(Xmat[:,k1*(topX+2)+1].astype(str))))
    # bacterial interactions
    for k2,c2 in enumerate(bacterial_taxa):
        v = list(Xmat[:,k1*(topX+2)+2+k2].astype(str))
        json_str += '\"pairwise_interaction_%s_%s\" : [%s]'%(c1,c2,','.join(v))
        if c1 == bacterial_taxa[-1] and c2 == bacterial_taxa[-1]:
            json_str += '\n}'
        else:
            json_str += ',\n'
#print(json_str)

text_file = open("/Users/liaoc/Documents/cmdstan-2.24.1/examples/mice_scfa/mice_scfa.data.json", "w")
text_file.write("%s" % json_str)
text_file.close()

## write model

In [13]:
# data block
model_str = 'data {\n'
model_str += '\tint<lower=0> N;\n'
model_str += '\tvector[N] dlogX;\n'
for c1 in bacterial_taxa:
    model_str += '\tvector[N] growth_rate_%s;\n'%(c1)
    model_str += '\tvector[N] rs_response_%s;\n'%(c1)
    for c2 in bacterial_taxa:
        model_str += '\tvector[N] pairwise_interaction_%s_%s;\n'%(c1,c2)
model_str += '}\n'

# parameter block
model_str += 'parameters {\n\treal<lower=0,upper=1> sigma;\n'
for c1 in bacterial_taxa:
    model_str += '\treal alpha__%s;\n'%(c1) # growth rate
    model_str += '\treal epsilon__%s;\n'%(c1) # resistance starch response
    for c2 in bacterial_taxa:
        model_str += '\treal beta__%s_%s;\n'%(c1,c2)
model_str += '}\n'       
        
# model block
model_str += 'model {\n\tsigma ~ uniform(0,1);\n'
for c1 in bacterial_taxa:
    model_str += '\talpha__%s ~ normal(0,1);\n'%(c1) # growth rate
    model_str += '\tepsilon__%s ~ normal(0,1);\n'%(c1) # rs response
    for c2 in bacterial_taxa:
        model_str += '\tbeta__%s_%s ~ normal(0,1);\n'%(c1,c2)
model_str += '\tdlogX ~ normal('
for c1 in bacterial_taxa:
    model_str += 'alpha__%s*growth_rate_%s+'%(c1,c1) # growth rate
    model_str += 'epsilon__%s*rs_response_%s+'%(c1,c1) # rs response
    for c2 in bacterial_taxa:
        if c1 == bacterial_taxa[-1] and c2 == bacterial_taxa[-1]:
            model_str += 'beta__%s_%s*pairwise_interaction_%s_%s'%(c1,c2,c1,c2)
        else:
            model_str += 'beta__%s_%s*pairwise_interaction_%s_%s+'%(c1,c2,c1,c2)
model_str += ', sigma);\n}'
#print(model_str)

text_file = open("/Users/liaoc/Documents/cmdstan-2.24.1/examples/mice_scfa/mice_scfa.stan", "w")
text_file.write("%s" % model_str)
text_file.close()

# Plot stan output

In [14]:
def hpd_grid(sample, alpha=0.05, roundto=2):
    """Calculate highest posterior density (HPD) of array for given alpha. 
    The HPD is the minimum width Bayesian credible interval (BCI). 
    The function works for multimodal distributions, returning more than one mode
    Parameters
    ----------
    
    sample : Numpy array or python list
        An array containing MCMC samples
    alpha : float
        Desired probability of type I error (defaults to 0.05)
    roundto: integer
        Number of digits after the decimal point for the results
    Returns
    ----------
    hpd: array with the lower 
          
    """
    sample = np.asarray(sample)
    sample = sample[~np.isnan(sample)]
    # get upper and lower bounds
    l = np.min(sample)
    u = np.max(sample)
    density = kde.gaussian_kde(sample)
    x = np.linspace(l, u, 2000)
    y = density.evaluate(x)
    #y = density.evaluate(x, l, u) waitting for PR to be accepted
    xy_zipped = zip(x, y/np.sum(y))
    xy = sorted(xy_zipped, key=lambda x: x[1], reverse=True)
    xy_cum_sum = 0
    hdv = []
    for val in xy:
        xy_cum_sum += val[1]
        hdv.append(val[0])
        if xy_cum_sum >= (1-alpha):
            break
    hdv.sort()
    diff = (u-l)/20  # differences of 5%
    hpd = []
    hpd.append(round(min(hdv), roundto))
    for i in range(1, len(hdv)):
        if hdv[i]-hdv[i-1] >= diff:
            hpd.append(round(hdv[i-1], roundto))
            hpd.append(round(hdv[i], roundto))
    hpd.append(round(max(hdv), roundto))
    ite = iter(hpd)
    hpd = list(zip(ite, ite))
    modes = []
    for value in hpd:
        x_hpd = x[(x > value[0]) & (x < value[1])]
        y_hpd = y[(x > value[0]) & (x < value[1])]
        modes.append(round(x_hpd[np.argmax(y_hpd)], roundto))
    return hpd, x, y, modes

In [17]:
fit = az.from_cmdstan(["output_%d.csv"%(i) for i in [1,2,3]])

In [18]:
lines = []
    
# basal growth rate
for c in bacterial_taxa:
    var = 'alpha__%s'%(c)
    data = []
    for i in np.arange(0,3):
        data.extend(list(fit.posterior[var][i].values))
    hpd_mu, x_mu, y_mu, modes_mu = hpd_grid(data)
    assert len(hpd_mu) == 1
    (x0, x1) = hpd_mu[0]
    lines.append(['basal_growth_rate', c, x0, x1, (x0+x1)/2, np.mean(data)/np.std(data), x0*x1>0])
    
# rs response
for c in bacterial_taxa:
    var = 'epsilon__%s'%(c)
    data = []
    for i in np.arange(0,3):
        data.extend(list(fit.posterior[var][i].values))
    hpd_mu, x_mu, y_mu, modes_mu = hpd_grid(data)
    assert len(hpd_mu) == 1
    (x0, x1) = hpd_mu[0]
    lines.append(['rs_response', c, x0, x1, (x0+x1)/2, np.mean(data)/np.std(data), x0*x1>0])

# pairwise interactions
for c1 in bacterial_taxa:
    for c2 in bacterial_taxa:
        var = 'beta__%s_%s'%(c1,c2)
        data = []
        for i in np.arange(0,3):
            data.extend(list(fit.posterior[var][i].values))
        hpd_mu, x_mu, y_mu, modes_mu = hpd_grid(data)
        assert len(hpd_mu) == 1
        (x0, x1) = hpd_mu[0]
        lines.append(['pairwise_interaction', (c1,c2), x0, x1, (x0+x1)/2, np.mean(data)/np.std(data), x0*x1>0])
    
df_stan_output_summary = pd.DataFrame(lines, columns = ['Type','Taxa','Left','Right','Middle','SNR','Significant'])
df_stan_output_summary.head()

,Type,Taxa,Left,Right,Middle,SNR,Significant
0,basal_growth_rate,Muribaculaceae,-0.15,0.24,0.045,0.443050,False
1,basal_growth_rate,Faecalibaculum,-0.18,0.22,0.020,0.209583,False
2,basal_growth_rate,Bacteroides_dash_acidifaciens,-0.34,0.04,-0.150,-1.542393,False
3,basal_growth_rate,Lachnospiraceae_dash_NK4A136_dash_group,-0.25,0.14,-0.055,-0.590692,False
4,basal_growth_rate,Parabacteroides_dash_goldsteinii,-0.36,0.03,-0.165,-1.638676,False


In [19]:
df_stan_output_summary.to_excel('bayesian_regression_summary_top20.xlsx')

# Plot

In [21]:
#df_stan_output_summary = pd.read_excel('bayesian_regression_summary_top20.xlsx', index_col=0)
df_stan_output_summary = df_stan_output_summary[df_stan_output_summary.Significant==True]
df_stan_output_summary

,Type,Taxa,Left,Right,Middle,SNR,Significant
20,rs_response,Muribaculaceae,0.42,0.90,0.660,5.491318,True
21,rs_response,Faecalibaculum,0.85,1.32,1.085,8.986253,True
27,rs_response,Alloprevotella,-0.49,-0.02,-0.255,-2.153641,True
29,rs_response,Desulfovibrionaceae,0.17,0.65,0.410,3.338960,True
40,pairwise_interaction,"(Muribaculaceae, Muribaculaceae)",2.88,4.31,3.595,9.897982,True
41,pairwise_interaction,"(Muribaculaceae, Faecalibaculum)",-4.73,-1.85,-3.290,-4.506180,True
43,pairwise_interaction,"(Muribaculaceae, Lachnospiraceae_dash_NK4A136_...",-5.85,-2.90,-4.375,-5.836662,True
48,pairwise_interaction,"(Muribaculaceae, Lachnospiraceae_dash_bacteriu...",-3.28,-0.39,-1.835,-2.509557,True
49,pairwise_interaction,"(Muribaculaceae, Desulfovibrionaceae)",-2.88,-0.11,-1.495,-2.125255,True
54,pairwise_interaction,"(Muribaculaceae, Lachnospiraceae_dash_bacteriu...",-3.75,-0.73,-2.240,-2.912772,True
